# Holdings List from csv export

In [1]:
import pandas as pd
import yfinance as yf
import yahooquery as yq

In [2]:
holdings = pd.read_csv('holdings.csv')

In [3]:
holdings = holdings[["Instrument", "Qty.", "Avg. cost"]]

In [4]:
symbols = holdings["Instrument"].to_list()
symbols = [ x if x.find("-") == -1 else x[:x.find("-")] for x in symbols]

In [5]:
def yfinance_equity_type(symbols: list[str]) -> (pd.Series, pd.Series):
    symbol = []
    eqtype = []
    for ticker in symbols:
        try:
            t = ticker + ".NS"
            data = yf.Ticker(t)
            if str(data.info["longName"]).find("ETF") != -1 or str(data.info["longName"]).find("Exchange Traded Fund") != -1:
                eqtype.append("ETF")
                symbol.append(t)
                continue
            eqtype.append("EQ")
            symbol.append(t)
        except:
            try:
                t = ticker + ".BO"
                data = yf.Ticker(t)
                data.info
                eqtype.append("EQ")
                symbol.append(t)
            except:
                eqtype.append("NF")
                symbol.append(ticker)
    return (pd.Series(eqtype, name="Type"), pd.Series(symbol, name="Symbol"))

def yahooquery_equity_type(symbols: list[str]) -> pd.Series:
    eqtype = []
    for ticker in symbols:
        try:
            t = ticker + ".NS"
            data = yq.Ticker(t)
            if type(data.summary_detail[t]) != type("xyz"):
                if type(data.all_financial_data()) == type("xyz") and str(data.all_financial_data()).find("data unavailable") != -1:
                    eqtype.append("ETF")
                    continue
                eqtype.append("EQ")
            else:
                t = ticker + ".BO"
                data = yq.Ticker(t)
                if type(data.summary_detail[t]) != type("xyz"):
                    eqtype.append("EQ")
                else:
                    raise Exception("check again")
        except:
            eqtype.append("NF")
    return pd.Series(eqtype, name="Type")

In [6]:
df = pd.DataFrame()
yet = yfinance_equity_type(symbols)
df["yfinance"] = yet[0]
df["Symbol"] = yet[1]
df["yahooquery"] = yahooquery_equity_type(symbols).copy()

In [7]:
maskeq = (df["yfinance"] == "EQ") & (df["yahooquery"] == "EQ")
masketf = (df["yfinance"] == "ETF") & (df["yahooquery"] == "ETF")
maskconf =  ((df["yfinance"] == "ETF") & (df["yahooquery"] != "ETF")) | (df["yfinance"] == "EQ") & (df["yahooquery"] != "EQ") | ((df["yfinance"] == "NF") & (df["yahooquery"] == "NF"))

In [8]:
df.loc[maskeq, ["Type"]] = "EQ"
df.loc[masketf, ["Type"]] = "ETF"
df.loc[maskconf, ["Type"]] = ""

In [9]:
holdings[["Symbol","yfType","yqType","Type"]] = df[["Symbol","yfinance","yahooquery","Type"]]

In [10]:
holdings.index.set_names("Sl. No.", inplace=True)

In [11]:
holdings.to_csv("holdtyped.csv")